# 3.4 softmax回归
- **目录**
  - 3.4.1 分类问题
  - 3.4.2 网络架构
  - 3.4.3 softmax运算
  - 3.4.4 小批量样本的矢量化
  - 3.4.5 损失函数
    - 3.4.5.1 对数似然
    - 3.4.5.2 softmax及其导数
    - 3.4.5.3 交叉熵损失
  - 3.4.6 信息论基础
    -  3.4.6.1 熵
    -  3.4.6.2 信息量
    -  3.4.6.3 重新审视交叉熵
  - 3.4.7 模型预测和评估

- **分类**问题：不是问“多少”，而是问“哪一个”：
  * 某个电子邮件是否属于垃圾邮件文件夹？
  * 某个用户可能“注册”或“不注册”订阅服务？
  * 某个图像描绘的是驴、狗、猫、还是鸡？
  * 某人接下来最有可能看哪部电影？
- 通常，机器学习实践者用*分类*这个词来描述两个有微妙差别的问题：
  - 只对样本的<b>“硬性”</b>类别感兴趣，即明确属于哪个类别；
  - 希望得到<b>“软性”</b>类别，即得到属于每个类别的**概率**。
- 这两者的界限往往很模糊。
  - 其中的一个原因是：**即使只是关心硬类别，仍可使用软类别的模型**。

------------
- **说明：如何理解“即使只是关心硬类别，仍可使用软类别的模型”？**
  - 在处理分类问题时，即便最终目标是确定每个样本的**确切类别（硬类别）**，仍然可以利用输出**概率分布（软类别）** 的模型来达到这一目的。
  - 这里的“硬类别”指的是最终的分类结果，例如将一个图像归类为“狗”，而“软类别”指的是对于每个类别的预测概率，例如模型预测该图像为狗的概率是80%，为猫的概率是15%，为鸡的概率是5%。
  - **软类别模型**：这类模型为每个可能的类别提供一个概率估计。
    - 例如，**softmax回归就是一个典型的软类别模型**，它可以输出一个概率分布，表明给定的输入样本属于每个类别的概率。
    - 模型的输出是一个**概率向量**，向量中的每个元素对应一个类别的概率。
  - **硬类别模型**：这类模型直接输出一个类别标签，也就是说，它执行的是一个明确的决策，没有提供额外的概率信息。
  - **从软类别到硬类别的转换**：即使只关心硬类别结果，软类别模型依然很有用，因为可以通过选择概率最高的类别来得到硬类别。
    - 例如，如果softmax回归模型给出的概率分布中，“狗”的类别概率最高，可以将该样本的硬类别标记为“狗”。
  - 使用软类别模型的理由：
    - **置信度**：软类别模型提供了关于预测置信度的信息。一个高概率的预测通常表示模型更加确定它的决策。反之，一个接近均匀分布的概率向量则表明模型对此不太确定。
    - **灵活性**：软类别模型更加灵活，可以用于各种不同的任务，包括那些需要概率估计的任务。
    - **决策阈值**：有时，可能会根据需要设置不同的阈值来做出决策，而不是简单地选择最高概率的类别。
    - **鲁棒性**：在某些情况下，即使在不确定的情况下，模型也能给出较为“柔和”的响应，这比硬性决策在实际应用中可能更鲁棒。
  - ImageNet数据集拥有大约1400万张图片，2000多个类别。
    - ILSVRC(ImageNet Large Scale Visual Recognition Challenge,ImageNet大规模视觉识别挑战赛,斯坦福大学李飞飞教授)使用 ImageNet 数据集的一个子集。
    - 该子集包含**1000个类别** ，每个类别约有 1000 张图像，共计约 120 万张训练图像、5 万张验证图像和 15 万张测试图像。
    - 2012 年，AlexNet 在 ImageNet 竞赛中取得了突破性成果，标志着深度学习时代的到来。
      - AlexNet 的最后一层是一个全连接层，它输出 1000 个类别的原始分数。
      - 然后将这些分数输入到 softmax 函数中，生成每个类别的概率分布。
------------

## 3.4.1 分类问题
- 假设每次输入是一个$2\times2$的灰度图像。
  - 可以用一个标量表示每个像素值，每个图像对应四个特征$x_1, x_2, x_3, x_4$。
  - 此外，假设每个图像属于类别“猫”，“鸡”和“狗”中的一个。
- 接下来，选择如何表示标签。
  - 有两个明显的选择：最直接的想法是选择$y \in \{1, 2, 3\}$，
其中整数分别代表$\{\text{狗}, \text{猫}, \text{鸡}\}$。
  - 这是在计算机上存储此类信息的有效方法。
- 如果类别间有一些**自然顺序**，比如预测$\{\text{婴儿}, \text{儿童}, \text{青少年}, \text{青年人}, \text{中年人}, \text{老年人}\}$，那么将这个问题转变为回归问题，并且保留这种格式是有意义的。
- 但是一般的分类问题**并不**与类别之间的**自然顺序**有关，有一种表示分类数据的简单方法——**独热编码（one-hot encoding）**：
  - 独热编码是一个向量，它的分量和类别一样多。
  - 类别对应的分量设置为1，其他所有分量设置为0。
  - 本例的标签$y$将是一个三维向量：
    - $(1, 0, 0)$对应于“猫”
    - $(0, 1, 0)$对应于“鸡”
    - $(0, 0, 1)$对应于“狗”：
$${y \in \{(1, 0, 0), (0, 1,0), (0, 0, 1)\}.\tag {3.4.1} }$$

## 3.4.2 网络架构

- 为了估计所有可能类别的条件概率需要一个有多个输出的模型，每个类别对应一个输出。
- 为了解决线性模型的分类问题，需要和输出一样多的**仿射函数（affine function）**,每个输出对应于它自己的仿射函数。
- 本例有4个**特征**和3个可能的**输出类别**，将需要12个标量来表示**权重**（带下标的$w$），3个标量来表示**偏置**（带下标的$b$），还有：$o_1$、$o_2$和$o_3$三个**未规范化**的预测（logit）输出：

$$
\begin{aligned}
o_1 &= x_1 w_{11} + x_2 w_{12} + x_3 w_{13} + x_4 w_{14} + b_1,\\
o_2 &= x_1 w_{21} + x_2 w_{22} + x_3 w_{23} + x_4 w_{24} + b_2,\\
o_3 &= x_1 w_{31} + x_2 w_{32} + x_3 w_{33} + x_4 w_{34} + b_3.
\end{aligned}
 \tag{3.4.2}$$

- 使用神经网络图来描述这个计算过程。与线性回归一样，**softmax回归也是一个单层神经网络**。
- 由于计算每个输出$o_1$、$o_2$和$o_3$取决于所有输入$x_1$、$x_2$、$x_3$和$x_4$，所以softmax回归的输出层也是**全连接层**。

<center><img src=../img/softmaxreg.svg /)</center>
<center>图3.4.1 softmax回归是一种单层神经网络</center>

- 通过向量形式表达为$\mathbf{o} = \mathbf{W} \mathbf{x} + \mathbf{b}$，
这是一种更适合数学和编写代码的形式。

- 由此，已经将所有权重放到一个$3 \times 4$矩阵中。
- 对于给定数据样本的特征$\mathbf{x}$，模型的输出是由权重与输入特征进行矩阵-向量乘法再加上偏置$\mathbf{b}$得到的。

## 3.4.3 softmax运算

- 模型的输出$\hat{y}_j$可被视为属于类$j$的概率，然后选择具有最大输出值的类别$\operatorname*{argmax}_j y_j$作为预测结果。
 - 例如，如果$\hat{y}_1$、$\hat{y}_2$和$\hat{y}_3$分别为0.1、0.8和0.1，那么模型预测的类别是2，在本例中代表“鸡”。

- 通常情况下不能将**未规范化**的预测$o$直接视作预测输出，因为将线性层的输出直接视为概率时存在一些问题：
  - 首先，没有限制这些输出数字的**总和为1**。
  - 其次，输出有可能为负值。
  - 二者都违反了概率基本公理。

- **softmax函数**：
  - softmax函数将未规范化的预测变换为非负并且总和为1，同时要求模型保持可导。
  - 首先对每个未规范化的预测求幂，这样可以确保输出非负。
  - 为了确保最终输出的总和为1，再对每个求幂后的结果除以它们的总和，如下式：

  $$\hat{\mathbf{y}} = \mathrm{softmax}(\mathbf{o})\quad \text{其中}\quad \hat{y}_j = \frac{\exp(o_j)}{\sum_k \exp(o_k)} \tag{3.4.3}$$
  - 这里，对于所有的$j$总有$0 \leq \hat{y}_j \leq 1$。因此，$\hat{\mathbf{y}}$可以视为一个正确的概率分布。
- softmax运算**不会改变**未规范化的预测$\mathbf{o}$之间的顺序，只会确定分配给每个类别的概率。
  - 因此，在预测过程中，仍然可以用下式来选择最有可能的类别。

$$
\operatorname*{argmax}_j \hat y_j = \operatorname*{argmax}_j o_j.
\tag{3.4.4}$$

- 最后有一个关键之处：尽管softmax是一个非线性函数，但softmax回归的输出仍然由输入特征的仿射变换决定。
因此，**softmax回归是一个线性模型**。

- **示例：softmax函数的计算**

In [1]:
import math
import numpy as np
def softmax(x):
    """
    实现 softmax 函数。

    参数:
        x (list): 一个一维列表，包含输入值。

    返回:
        list: 一个一维列表，包含经过 softmax 变换后的概率分布。
    """
    # 计算所有输入值的指数
    exps = np.exp(x) 

    # 计算输入值指数之和
    sum_exps = np.sum(exps, axis = 1, keepdims=True)

    # 对每个输入值的指数进行归一化
    result = exps / sum_exps

    return result

# 示例输入值
input_values =np.array([[1.0, 2.0, 3.0],[6.0, 3.0, 9.0],[11.0, 3.0, 7.0]])

# 使用自定义 softmax 函数计算概率分布
output_values = softmax(input_values)

# 使用固定点表示法来打印浮点数
np.set_printoptions(suppress=True) 
print("输入值:", input_values)
print("Softmax 输出概率分布:", output_values)
print("输出概率分布之和:", np.sum(output_values, axis = 1))

输入值: [[ 1.  2.  3.]
 [ 6.  3.  9.]
 [11.  3.  7.]]
Softmax 输出概率分布: [[0.09003057 0.24472847 0.66524096]
 [0.04731416 0.00235563 0.95033021]
 [0.98169039 0.00032932 0.01798029]]
输出概率分布之和: [1. 1. 1.]


## 3.4.4 小批量样本的矢量化

- 为了提高计算效率并且充分利用GPU，通常会针对**小批量**数据执行**矢量计算**。
  - 假设读取了一个批量的样本$\mathbf{X}$，其中特征维度（输入数量）为$d$，批量大小为$n$。
  - 假设在输出中有$q$个类别。那么小批量特征为$\mathbf{X} \in \mathbb{R}^{n \times d}$，
权重为$\mathbf{W} \in \mathbb{R}^{d \times q}$，偏置为$\mathbf{b} \in \mathbb{R}^{1\times q}$。
  - softmax回归的矢量计算表达式为：

 $$ \begin{aligned} \mathbf{O} &= \mathbf{X} \mathbf{W} + \mathbf{b}, \\ \hat{\mathbf{Y}} & = \mathrm{softmax}(\mathbf{O}). \end{aligned} \tag{3.4.5}$$


- 相对于一次处理一个样本，小批量样本的矢量化加快了$\mathbf{X}$和$\mathbf{W}$的矩阵-向量乘法：
  - 由于$\mathbf{X}$中的每一行代表一个数据样本，那么softmax运算可以**按行（rowwise）** 执行：对于$\mathbf{O}$的每一行，先对所有项进行<b>幂运算</b>，然后通过求和对它们进行标准化。
  - 所谓标准化就是将每项除以各项之和。
  - $\mathbf{X} \mathbf{W} + \mathbf{b}$的求和会使用广播，小批量的未规范化预测$\mathbf{O}$和输出概率$\hat{\mathbf{Y}}$
都是形状为$n \times q$的矩阵。
- **示例：**

In [2]:
import numpy as np

# 初始化输入特征矩阵 X, 权重矩阵 W 和 偏置向量 b
n, d, q = 5, 4, 3  # 例如，批量大小为5，特征维度为4，类别数量为3
np.random.seed(0)  # 为了可重复性
X = np.random.randn(n, d)
W = np.random.randn(d, q)
b = np.random.randn(1, q)

# 计算未规范化的预测 O
O = np.dot(X, W) + b

# 定义 softmax 函数
def softmax(matrix):
    exp_matrix = np.exp(matrix - np.max(matrix, axis=1, keepdims=True))  # 稳定计算 softmax    
    return exp_matrix / np.sum(exp_matrix, axis=1, keepdims=True)

# 计算输出概率 Y_hat
Y_hat = softmax(O)

print("输入特征矩阵 X:\n", X)
print("权重矩阵 W:\n", W)
print("偏置向量 b:\n", b)
print("未规范化的预测 O:\n", O)
print("输出概率 Y_hat:\n", Y_hat)


输入特征矩阵 X:
 [[ 1.76405235  0.40015721  0.97873798  2.2408932 ]
 [ 1.86755799 -0.97727788  0.95008842 -0.15135721]
 [-0.10321885  0.4105985   0.14404357  1.45427351]
 [ 0.76103773  0.12167502  0.44386323  0.33367433]
 [ 1.49407907 -0.20515826  0.3130677  -0.85409574]]
权重矩阵 W:
 [[-2.55298982  0.6536186   0.8644362 ]
 [-0.74216502  2.26975462 -1.45436567]
 [ 0.04575852 -0.18718385  1.53277921]
 [ 1.46935877  0.15494743  0.37816252]]
偏置向量 b:
 [[-0.88778575 -1.98079647 -0.34791215]]
未规范化的预测 O:
 [[-2.35091443  0.24449631  2.9426347 ]
 [-5.10926422 -3.17960044  4.08683014]
 [ 1.21443984 -0.91793107 -0.2635599 ]
 [-2.41041243 -1.23857768  0.9395239 ]
 [-5.79054067 -1.66083867  1.39887569]]
输出概率 Y_hat:
 [[0.00468492 0.06278775 0.93252732]
 [0.00010135 0.00069804 0.9992006 ]
 [0.7425838  0.08803767 0.16937852]
 [0.0305541  0.09862588 0.87082001]
 [0.00072021 0.04476766 0.95451213]]


In [20]:
# “不”稳定性操作结果是一样的，至少在本例中是一样的
np.exp(O),np.sum(np.exp(O),axis=1,keepdims=True), np.exp(O)/np.sum(np.exp(O),axis=1,keepdims=True)

(array([[ 0.09528199,  1.27697795, 18.96574967],
        [ 0.00604053,  0.04160227, 59.55082436],
        [ 3.36840667,  0.3993444 ,  0.7683116 ],
        [ 0.08977826,  0.28979611,  2.55876291],
        [ 0.00305633,  0.18997958,  4.05064323]]),
 array([[20.33800962],
        [59.59846716],
        [ 4.53606267],
        [ 2.93833728],
        [ 4.24367914]]),
 array([[0.00468492, 0.06278775, 0.93252732],
        [0.00010135, 0.00069804, 0.9992006 ],
        [0.7425838 , 0.08803767, 0.16937852],
        [0.0305541 , 0.09862588, 0.87082001],
        [0.00072021, 0.04476766, 0.95451213]]))

## 3.4.5 损失函数
- 在线性回归中，一般需要使用某种方法度量损失函数预测的效果，此处一般使用最大似然估计。

### 3.4.5.1 对数似然
- softmax函数给出了一个向量$\hat{\mathbf{y}}$，
可以将其视为“对给定任意输入$\mathbf{x}$的每个类的条件概率”。
- 例如，$\hat{y}_1$=$P(y=\text{猫} \mid \mathbf{x})$。
假设整个数据集$\{\mathbf{X}, \mathbf{Y}\}$具有$n$个样本，
其中索引$i$的样本由特征向量$\mathbf{x}^{(i)}$和独热标签向量$\mathbf{y}^{(i)}$组成。
可以将估计值与实际值进行比较：
$$P(\mathbf{Y} \mid \mathbf{X}) = \prod_{i=1}^n P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)}) \tag{3.4.6}$$

- 根据最大似然估计，可最大化$P(\mathbf{Y} \mid \mathbf{X})$，相当于最小化负对数似然：
 $$
-\log P(\mathbf{Y} \mid \mathbf{X}) = \sum_{i=1}^n -\log P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)})
= \sum_{i=1}^n l(\mathbf{y}^{(i)}, \hat{\mathbf{y}}^{(i)}) \tag{3.4.7}$$    
- 其中，对于任何标签$\mathbf{y}$和模型预测$\hat{\mathbf{y}}$，损失函数为：
 $$ l(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{j=1}^q y_j \log \hat{y}_j \tag{3.4.8}$$该损失函数通常被称为**交叉熵损失（cross-entropy loss）**。
- 由于$\mathbf{y}$是一个长度为$q$的独热编码向量，所以除了一个项以外的所有项$j$都消失了。
- 由于所有$\hat{y}_j$都是预测的概率，所以它们的对数永远不会大于$0$。


### 3.4.5.2 softmax及其导数
由于softmax和相关的损失函数很常见，因此需要更好地理解它的计算方式。
- 将 (3.4.3)代入损失 (3.4.8)中，利用softmax的定义，可得：
$$
\begin{aligned}
l(\mathbf{y}, \hat{\mathbf{y}}) &=  - \sum_{j=1}^q y_j \log \frac{\exp(o_j)}{\sum_{k=1}^q \exp(o_k)} \\
&= \sum_{j=1}^q y_j \log \sum_{k=1}^q \exp(o_k) - \sum_{j=1}^q y_j o_j\\
&= \log \sum_{k=1}^q \exp(o_k) - \sum_{j=1}^q y_j o_j.
\end{aligned}
\tag{3.4.9}$$

In [3]:
# 公式3.4.9中第2步到第3步推导示例，减号前面部分
y_onehot = np.array([0,1,0]) # 独热编码向量，模拟y_i求和
o_k = np.array([0.3,0.4,0.9]) # 模拟内层log表达式的求和数据
# 首先求两层循环的求和运算
sum1 = 0
for i in y_onehot:
    sum1 += i*np.sum(o_k)
# 只对内层循环求和
sum2 = np.sum(o_k)
# 两种求法结果相同
sum1,sum2

(1.6, 1.6)

In [4]:
# 公式3.4.9中第2步到第3步推导示例，减号后面部分
sum3 = np.zeros(3)
for i in y_onehot:
    sum3[i] = y_onehot[i]*o_k[i] 
sum3,  y_onehot*o_k

(array([0. , 0.4, 0. ]), array([0. , 0.4, 0. ]))

- 考虑相对于任何未规范化的预测$o_j$的导数，得到：

$$
\partial_{o_j} l(\mathbf{y}, \hat{\mathbf{y}}) = \frac{\exp(o_j)}{\sum_{k=1}^q \exp(o_k)} - y_j = \mathrm{softmax}(o_j) - y_j = {\hat y}_j - y_j.
\tag{3.4.10}$$

- 换句话说，导数是**softmax模型分配的概率与实际发生的情况（由独热标签向量表示）之间的差异**。
- 从这个意义上讲，这与回归中看到的非常相似，**其中梯度是观测值$y$和估计值$\hat{y}$之间的差异**。

- （**补充**）下一步根据链式法则继续求${\mathbf W}$和${\mathbf b}$的梯度:
$$
\partial_{W} l(\mathbf{y}, \hat{\mathbf{y}}) = {\partial_{o} l(\mathbf{y}, \hat{\mathbf{y}})} \partial_{W} o =  \mathbf X^T({\hat Y} - Y) \tag {补充公式1}$$
$$\partial_{b} l(\mathbf{y}, \hat{\mathbf{y}}) = {\partial_{o} l(\mathbf{y}, \hat{\mathbf{y}})} \partial_{b} o =  \sum_{i=1}^n({\hat y}^{(i)} - y^{(i)}) \tag {补充公式2}$$

### 3.4.5.3 交叉熵损失

- 考虑整个结果分布的情况，即观察到的不仅仅是一个结果。
- 对于标签$\mathbf{y}$，可以使用与以前相同的表示形式。
- 唯一的区别是，现在用一个概率向量表示，如$(0.1, 0.2, 0.7)$，
而不是仅包含**二元项**的向量$(0, 0, 1)$。
- 使用公式$3.4.8$来定义损失$l$，它是所有标签分布的预期损失值。
- 此损失称为**交叉熵损失（cross-entropy loss）**，它是分类问题最常用的损失之一。


- **示例：softmax函数以及交叉熵损失在分类任务中应用**

In [5]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

'''
Softmax回归
'''
class SoftmaxRegression:
    def __init__(self, learning_rate=0.1, epochs=1000):
        self.learning_rate = learning_rate
        self.epochs = epochs
     
    # 拟合函数
    def fit(self, X, y):
        # 样本数（本例为150 * 0.8 = 120），特征数（本例为4）
        n_samples, n_features = X.shape
        # 分类类型数（本例为3）
        n_classes = len(np.unique(y))
        
        # 初始化权重矩阵，形状为(4, 3)
        self.W = np.zeros((n_features, n_classes))
        # 初始化偏置，形状为(3,)
        self.b = np.zeros(n_classes)
        
        # one-hot编码，形状为(120, 3)
        Y_onehot = np.zeros((n_samples, n_classes))
        '''
        鸢尾花有三个类型，因此其独热编码是类似：[1, 0, 0],[0, 1, 0],[0, 0, 1]。
        y作为分类编码分别为0, 1, 2三个值之一，
        然后Y_onehot的120行中每行以y作为列索引的相应列设为1，其他列为0。
        Y_onehot的数据形状类似：
        [[0,0,1]
         [1,0,0]
         [0,1,0]
         ...
         [0,0,1]]
        '''
        Y_onehot[np.arange(n_samples), y] = 1
        
        # 训练模型
        for i in range(self.epochs):
            # 打分是线性模型
            # (120,4)@(4,3)+(120,3)=(120, 3)
            # 注意self.b形状本位(3,)经广播后变成(120,3)
            scores = np.dot(X, self.W) + self.b
            # 以自然常数e为底的幂运算，形状不变
            exp_scores = np.exp(scores)
            
            '''
            计算softmax函数值，axis=1是计算行向量的和。
            因此此处的概率是计算每一行的幂打分除以每一行的幂打分之和。
            也就是每一个样本的softmax函数值,计算结果仍然是(120, 3)。
            但是每一行的各列数据都小于等于1且大于等于0，而且每一行3个数据之和等于1。
            probs的数据如下所示，一共120行：
            [[0.09003057 0.24472847 0.66524096]
             [0.04731416 0.00235563 0.95033021]
             [0.98169039 0.00032932 0.01798029]
             ...
            ]
            '''
            probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
            
            # 梯度下降
            error = probs - Y_onehot
            # W和b的梯度
            # dw = (120, 4).T@(120, 3) = (4, 120)@(120, 3) = (4, 3)
            # 此处计算W权重参数的梯度或偏导数可参见3.4.6.2节的(补充公式1)
            dW = np.dot(X.T, error)
            # db = (1, 3)
            db = np.sum(error, axis=0)
            # 梯度下降算法，更新W和b参数
            #print('-------------\n',dW,'\n******************\n',db)
            self.W -= self.learning_rate * dW
            self.b -= self.learning_rate * db
            
    def predict(self, X):
        scores = np.dot(X, self.W) + self.b
        # 通过argmax计算预测结果，仍是以0, 1, 2形式表示类别
        preds = np.argmax(scores, axis=1)
        return preds

# 加载数据集
iris = load_iris()
X = iris.data
y = iris.target

# 划分训练集和测试集
# 总共150条数据，划分成120条训练数据，30条测试数据
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 训练模型并预测
model = SoftmaxRegression(learning_rate=0.1, epochs=1000)
model.fit(X_train, y_train)
preds = model.predict(X_test)

# 计算准确率
acc = np.mean(preds == y_test)
print("Accuracy:", acc)


Accuracy: 0.9666666666666667


## 3.4.6 信息论基础
- **信息论（information theory）** 涉及编码、解码、发送以及尽可能简洁地处理信息或数据。

### 3.4.6.1 熵

- 信息论的核心思想是**量化**数据中的信息内容。
- 在信息论中，该数值被称为分布$P$的**熵（entropy）**，公示如下：
$$H[P] = \sum_x - P(x) \log P(x).\tag{3.4.11}$$
- 信息论的基本定理之一指出，为了对从分布$p$中随机抽取的数据进行编码，至少需要$H[P]$“**纳特（nat）**”对其进行编码。
- “纳特”相当于**比特（bit）**，但是对数底为$e$而不是2。
- 因此，一个纳特是$\frac{1}{\log(2)} \approx 1.44$比特。

### 3.4.6.2 信息量

- 如果不能完全预测每一个事件，那么有时可能会感到"**惊异（或惊奇，Surprisal）**"。
  - 惊异是用来衡量某个事件出现的**意外程度**的概念。
  - 在信息论中，每个事件都有一个对应的概率值，如果这个事件的概率值很小，那么当它发生时就会感到更加意外和惊异。
- 克劳德·香农决定用信息量$\log \frac{1}{P(x)} = -\log P(x)$来量化这种惊异程度。
  - 在观察一个事件$x$时，并赋予它（主观）概率$P(x)$。
  - 当赋予一个事件较低的概率时，惊异会更大，该事件的信息量也就更大。
- 所谓**熵**，是当分配的概率真正匹配数据生成过程时的**信息量的期望**。

### 3.4.6.3 重新审视交叉熵

- 如果把熵$H(P)$想象为“知道真实概率的人所经历的惊异(或惊奇)程度”？
- **交叉熵**从$P$ 到$Q$，记为$H(P, Q)$。

  $$
  H(P, Q) = -\sum_{x} P(x) \log Q(x)\tag{3.4.12}
  $$
- 当 $ P $ 是真实分布而 $ Q $ 是模型的预测分布时，交叉熵可以理解为在模型预测分布 $ Q $ 下，观察到真实数据分布 $ P $ 时的平均惊异程度。
- 交叉熵的最小化。
  - 希望模型的预测分布 $ Q $ 尽可能接近真实分布 $ P $，这样可以最小化传达真实标签所需的平均惊异程度。这可以通过最小化交叉熵来实现。
- 最小化交叉熵：
  $$
  \hat{Q} = \arg\min_{Q} H(P, Q) \tag{3.4.13}
  $$
- 当 $ Q $ 趋近于 $ P $ 时，交叉熵 $ H(P, Q) $ 也趋近于熵 $ H(P) $，即：
  $$
  H(P, P) = -\sum_{x} P(x) \log P(x) \tag{3.4.14}
  $$
- 在这种情况下，传达真实标签所需的惊异程度就是系统的固有不确定性（熵）。
- 结合两者：
  - 最大化观测数据的似然和最小化传达标签所需的惊异两者实际上是一回事，都是通过调整模型参数，使得模型的预测分布尽可能接近真实分布。
- 具体来说：
  - **最大化观测数据的似然**：通过最大化对数似然函数，我们调整模型参数，使得在给定参数下观测数据出现的概率最大。
  - **最小化传达标签所需的惊异**：通过最小化交叉熵损失，我们调整模型参数，使得模型的预测分布与真实分布之间的差异最小，从而减少传达真实标签所需的惊异程度。

- 可将交叉熵想象为“<b>主观概率为$Q$的观察者在看到根据概率$P$生成的数据时的预期惊异</b>”。
- 当$P=Q$时，交叉熵达到**最低**。
在这种情况下，从$P$到$Q$的交叉熵是$H(P, P)= H(P)$。
- 简而言之，可从两方面来考虑交叉熵分类目标：
  - （i）最大化观测数据的似然；
  - （ii）最小化传达标签所需的惊异。
- **示例：惊异的计算**

In [6]:
import math

# 定义一个函数来计算惊异值
def surprisal(prob):
    return -math.log2(prob)

# 示例：投掷一枚硬币，正反面概率相等
p = 0.5
s = surprisal(p)
print("投掷一枚硬币，正反面概率相等时的惊异值为：", s)

# 示例：摇骰子，点数为1到6的概率均等
p = 1/6
s = surprisal(p)
print("摇骰子，点数为1到6的概率均等时的惊异值为：", s)

# 示例：中了彩票，中奖概率非常小
p = 1/1000000
s = surprisal(p)
print("中了彩票，惊异值为：", s)


投掷一枚硬币，正反面概率相等时的惊异值为： 1.0
摇骰子，点数为1到6的概率均等时的惊异值为： 2.584962500721156
中了彩票，惊异值为： 19.931568569324174


## 3.4.7 模型预测和评估

- 在训练softmax回归模型后，给出任何样本特征，可以预测每个输出类别的概率。
  - 通常使用预测概率最高的类别作为输出类别。
  - 如果预测与实际类别（标签）一致，则预测是正确的。
- 在接下来的实验中，将使用**精度（accuracy）** 来评估模型的性能。
  - 精度等于正确预测数与预测总数之间的比率。

## 小结

* softmax运算获取一个向量并将其映射为概率。
* softmax回归适用于分类问题，它使用了softmax运算中输出类别的概率分布。
* 交叉熵是一个衡量两个概率分布之间差异的很好的度量，它测量给定模型编码数据所需的比特数。

